In [2]:
import pandas as pd
import numpy as np
import os
import healpy as hp
from astroquery.vizier import Vizier
from astroquery.gaia import Gaia
#import astropy.coordinates as coord
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.time import Time
import math
from astropy.io import ascii

'''Set of functions for getting a array of star list from [radsec]-area around the object, 
transformation the information to common view according [info] and writting into DataFrame.'''



def RenameColumns(table, catalogue):
    global info
    
    old_columns = info.drop(['link', 'pecularity'], axis = 1)
    new_columns = info.drop(['link', 'pecularity'], axis = 1)
    new_columns = new_columns.rename(columns = {'phot_g_mean_mag': 'mag'})
        
    old_columns = old_columns.loc[catalogue].tolist()
    new_columns = new_columns.columns.tolist()
            
    for i in range(len(new_columns)):
        table = table.rename(columns= {old_columns[i]: new_columns[i]})
    return table

def errors_2MASS(row):
    a = row['errMaj']
    b = row['errMin']
    angle = row['errPA']
    row['errMaj'] = abs(a * b) / (math.sqrt(
        (a ** 2) * (math.cos(math.radians(angle)) ** 2) + 
        (b ** 2) * (math.sin(math.radians(angle)) ** 2)))
    row['errMin'] = abs(a * b) / (math.sqrt(
        (a ** 2) * (math.sin(math.radians(angle)) ** 2) + 
        (b ** 2) * (math.cos(math.radians(angle)) ** 2)))
    return row

def epoch_UCAC4(row):
    a = row['EpRA']
    b = row['EpDE']
    row['EpRA'] = (a + b)/2
    return row

def CatDataFrame(star, catalogue):
    global info
    global radsec
    try:
        link = info.loc[catalogue]['link']
        viz = Vizier(columns=['all'])
        table = viz.query_region(SkyCoord(
            ra=star.cat_ra, dec=star.cat_dec, unit=(u.deg, u.deg), frame='icrs'),
                                 radius=radsec*u.arcsec, catalog=link)[0]
        
        df = pd.DataFrame.from_records(table.as_array())
        
        pecularities = info.pecularity[catalogue]
        pecs = pecularities.split('|')
            
        if (not pecs[0] == 'mas'):
                
            column1 = info ['ra_error'][catalogue]
            column2 = info ['dec_error'][catalogue]
                    
            df[column1] = df[column1].apply(lambda x: x*1000)
            df[column2] = df[column2].apply(lambda x: x*1000)
                    
            if len(pecs[0].split(':')) == 2:
                df[['errMaj', 'errMin', 'errPA']] = \
                df[['errMaj', 'errMin', 'errPA']].apply(errors_2MASS, axis = 1)
                        
                
        if (not pecs[1] == 'jy'):
            ps = pecs[1].split(':')
                    
            if len(ps) == 2:
                column = info ['ref_epoch'][catalogue]
                
                df[column] = \
                df[column].apply(lambda x: Time((x+ int(ps[0])), format=ps[1]).jyear)
                
            else:
                df[['EpRA','EpDE']] = \
                df[['EpRA','EpDE']].apply(epoch_UCAC4, axis = 1)

        cols = info.loc[catalogue].tolist()[:-2]
        df = df[['_r'] + cols]
        df = RenameColumns(df, catalogue)
        #df['delta_mag'] = df.mag
        #df.delta_mag = df.delta_mag.apply(lambda x: abs(x-star.mag))
        #first_col = df.pop('delta_mag')
        #df.insert(0, 'delta_mag', first_col)
        #df.delta_mag = round(df.delta_mag, 2)
        df._r = round(df._r, 4)
        #df.delta_mag = round(df.delta_mag, 2)
        df.ref_epoch = round(df.ref_epoch, 1)
        
        print(catalogue)# + ': success')
        return [df, True]
    
    except Exception as e:
        print(catalogue + '\t' + ' ERROR:', e)
        return ['NaN', False]
    
    
'''To find a star in array.'''
def best_match(df):
    global radsec
    global min_delta_mag
    number = 0
    if len(df) > 1:
        no_mag = True
        minimal = min_delta_mag
        for i in range(len(df)):
            s = df.iloc[i]
            if s.delta_mag < minimal:
                no_mag = False
                minimal = s.delta_mag
                number = i
        if no_mag:
            distance = radsec
            for i in range(len(df)):
                s = df.iloc[i]
                if s._r < distance:
                    distance = s._r
                    number = i
    s = df.drop('mag', axis = 1).iloc[number].apply(str)
    list_s = s.tolist()
    return '|'.join(list_s)

'''Set of functions for Mu and F calculations.'''

def tangFromRADE(ra, de, RA, DE):
    ksi = math.cos(de)*math.sin(ra-RA)/(math.sin(de)*math.sin(DE)+math.cos(de)*math.cos(DE)*math.cos(ra-RA))
    eta = (math.sin(de)*math.cos(DE)-math.cos(de)*math.sin(DE)*math.cos(ra-RA))/(math.sin(de)*math.sin(DE)+math.cos(de)*math.cos(DE)*math.cos(ra-RA))
    return ksi,eta


def RADecFromTang(ksi, eta, RA, Dec):
    secRho = math.sqrt(1+ksi*ksi+eta*eta)
    t11 = -math.sin(RA)
    t12 = math.cos(RA)
    t13 = 0
    t21 = -math.cos(RA)*math.sin(Dec)
    t22 = -math.sin(RA)*math.sin(Dec)
    t23 = math.cos(Dec)
    t31 = math.cos(RA)*math.cos(Dec)
    t32 = math.sin(RA)*math.cos(Dec)
    t33 = math.sin(Dec)
    x = (ksi*t11+eta*t21+t31)/secRho
    y = (ksi*t12+eta*t22+t32)/secRho
    z = (ksi*t13+eta*t23+t33)/secRho
    ra = math.atan2(y,x)
    dec = math.atan2(z,math.sqrt(x*x+y*y))
    if(ra<0):
        ra+=2*math.pi
    return ra,dec

def calibration(x,y,ksi,eta,n,Nmin,rmax):
    Q = int((n+1)*(n+2)/2)
    C = np.ones((np.size(x), Q))
    q = 0
    for i in range(n + 1):
        for j in range(n + 1):
            if (i + j <= n):
                C[:, q] = (x ** i) * (y ** j)
                q += 1
    We = np.diag(np.ones(np.size(x)))
    flag = 0
    it = 0
    while (flag == 0):
        Zx = np.dot(np.linalg.inv(np.dot(np.dot(np.transpose(C), We), C)),\
                    np.dot(np.dot(np.transpose(C), We), ksi))
        Zy = np.dot(np.linalg.inv(np.dot(np.dot(np.transpose(C), We), C)),\
                    np.dot(np.dot(np.transpose(C), We), eta))
        rx = np.dot(We, ksi - np.dot(C, Zx))
        ry = np.dot(We, eta - np.dot(C, Zy))
        r = np.sqrt(rx ** 2 + ry ** 2)
        kmax = np.argmax(r)
        flag = 1
        if (np.size(ksi) - it <= Nmin):
            break
        if (r[kmax] > rmax):
            We[kmax, kmax] = 0
            flag = 0
            it += 1
    uwex = np.dot(np.dot(np.transpose(rx), We), rx) / (np.size(x) - it - Q)
    uwey = np.dot(np.dot(np.transpose(ry), We), ry) / (np.size(y) - it - Q)
    return Zx,Zy,math.sqrt(uwex),math.sqrt(uwey),np.size(x)-it


def transform(x,y,Z):
    Q = np.size(Z)
    n = int(np.max(np.roots(np.array([1,3,-2*(Q-1)]))))
    C = np.ones((np.size(x), Q))
    q = 0
    for i in range(n+1):
        for j in range(n+1):
            if (i + j <= n):
                C[:, q] = (x ** i) * (y ** j)
                q += 1
    return np.dot(C,Z)


def sqrerror(e0, e, tg, tc):
    e2 = e / (tg - tc)
    e2 = e2**2 + e0**2
    #e - error from calibration, e0 - pm error from gaia, tg - gaia epoch, tc - cat epoch
    return e2

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


In [ ]:
def GaiaDataFrame(star):
    global radsec
    global info
    global limmag
    job = Gaia.launch_job_async("SELECT * FROM gaiadr2.gaia_source \
    WHERE CONTAINS(POINT('ICRS',gaiadr2.gaia_source.ra,gaiadr2.gaia_source.dec), \
    CIRCLE('ICRS',%f,%f,%f))=1 AND  phot_g_mean_mag<%f \
    AND pmra IS NOT NULL AND abs(pmra)>0 AND pmdec IS NOT NULL AND abs(pmdec)>0;" \
                                % (ra0, dec0, fov, limgmag) , dump_to_file=False)
    gaiat = job.get_results()
    

In [48]:
'''Global variables'''

radsec = 600
min_delta_mag = 10
limmag = 18

info = pd.read_csv('cat_info.csv').fillna('NaN')
info

,ra,dec,ra_error,dec_error,ref_epoch,phot_g_mean_mag,link,pecularity
CMC15,RA_ICRS,DE_ICRS,e_RA_ICRS,e_DE_ICRS,MJD-51263,r_mag,I/327/cmc15,arcsec|51263:mjd
Pan-STARRS DR1,RAJ2000,DEJ2000,e_RAJ2000,e_DEJ2000,_tab1_10,gmag,II/349/ps1,arcsec|0:mjd
2MASS All-Sky,RAJ2000,DEJ2000,errMaj,errMin,_tab1_36,Jmag,II/246/out,arcsec:errPA|0:jd
SDSS DR12,RA_ICRS,DE_ICRS,e_RA_ICRS,e_RA_ICRS,ObsDate,gmag,V/147/sdss12,arcsec|jy
URAT1,RAJ2000,DEJ2000,sigs,sigm,Epoch,gmag,I/329/urat1,mas|jy
UCAC4,RAJ2000,DEJ2000,e_RAJ2000,e_DEJ2000,EpRA,gmag,I/322A/out,mas|EpDE


In [2]:
directory = 'catalog_positions'
files = os.listdir(directory)
files = ['Pan-STARRS DR1.csv',
 'URAT1.csv',
 '2MASS All-Sky.csv',
 'SDSS DR12.csv',
 'CMC15.csv',
 'UCAC4.csv']

In [40]:
file = files[1]
catname = file.split('.')[0]
filepath = os.path.join(directory, file)

In [41]:
df = pd.read_csv(filepath, index_col = 0)
display(catname, df.head())

'URAT1'

,gaia_ra,gaia_dec,gaia_ra_error,gaia_dec_error,pmra,pmdec,pmra_error,pmdec_error,gaia_epoch,cat_ra,cat_dec,cat_epoch,delta_mag,dist_sec
source_id,,,,,,,,,,,,,,
3999433452516434560,180.185765,20.848187,0.047755,0.028939,-129.496727,235.954725,0.104269,0.060966,2015.5,180.185833,20.848086,2013.9,1.67,0.414
3999432971480096640,180.136496,20.815287,0.133700,0.092837,-130.134956,235.456935,0.269611,0.209330,2015.5,180.136574,20.815167,2013.6,NaN,0.490
2068138392112597376,308.009644,41.926820,0.630414,1.005075,21.121711,-5.906501,1.250776,1.766670,2015.5,308.017775,41.928574,2013.5,NaN,22.651
2288204854134519424,295.018022,73.911317,0.342117,0.353923,-23.876102,296.312861,0.765137,0.720155,2015.5,295.022253,73.914252,2013.8,NaN,11.357
2877522230262577024,2.169965,36.626590,0.037719,0.022303,-114.068699,-124.300389,0.055845,0.033946,2015.5,2.170056,36.626655,2013.5,NaN,0.325


In [42]:
debugging = 0

for s in list(df.index):
    
    debugging +=1
    if debugging > 5:
        break
    
    star = df.loc[s]

In [43]:
star = df.loc[3999433452516434560]

In [47]:
CatDataFrame(star, catname)

URAT1


[         _r          ra        dec  ra_error  dec_error  ref_epoch        mag
 0   571.290  180.228408  20.694465       164        108     2014.2        NaN
 1   575.555  180.228893  20.693359        48         43     2013.5        NaN
 2   572.863  180.195857  20.689231        71         43     2013.3        NaN
 3   519.097  180.208423  20.705444        47         43     2013.6        NaN
 4   516.899  180.177637  20.704705         9         14     2013.5        NaN
 5   497.533  180.272287  20.735974        26         42     2013.7        NaN
 6   558.842  180.292238  20.728909        21         28     2013.6        NaN
 7   437.860  180.205478  20.727850        14          8     2013.7        NaN
 8   457.864  180.219170  20.724776        21         25     2013.4        NaN
 9   443.733  180.234352  20.733473         7          6     2013.8  15.478000
 10  430.774  180.242525  20.740799        91         49     2014.1        NaN
 11  428.418  180.194163  20.729333        12       

In [59]:
job = Gaia.launch_job_async("SELECT * FROM gaiadr2.gaia_source \
WHERE CONTAINS(POINT('ICRS',gaiadr2.gaia_source.ra,gaiadr2.gaia_source.dec), \
CIRCLE('ICRS',%f,%f,%f))=1 AND  phot_g_mean_mag<%f AND pmra IS NOT NULL AND abs(pmra)>0 AND pmdec IS NOT NULL AND abs(pmdec)>0;" \
                            % (star.gaia_ra, star.gaia_dec, radsec, limmag) , dump_to_file=False)
gaiat = job.get_results()

SSLCertVerificationError: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1056)

In [56]:
job = Gaia.launch_job_async("SELECT * \
FROM gaiadr2.gaia_source \
WHERE CONTAINS(POINT('ICRS',gaiadr2.gaia_source.ra,gaiadr2.gaia_source.dec),CIRCLE('ICRS',%f,%f,%f))=1 \
AND pmra IS NOT NULL AND abs(pmra)>0 \
AND pmdec IS NOT NULL AND abs(pmdec)>0 \
AND  phot_g_mean_mag<%f;" % (star.gaia_ra, star.gaia_dec, radsec, limmag) \
                            , dump_to_file=False)

SSLCertVerificationError: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1056)

In [62]:
coord = SkyCoord(ra = star.gaia_ra, dec = star.gaia_dec, unit=(u.degree, u.degree), frame='icrs')
radius = u.Quantity(radsec, u.arcsec)
j = Gaia.cone_search_async(coord, radius)
r = j.get_results()
r


SSLCertVerificationError: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1056)

In [63]:
coord = SkyCoord(ra = star.gaia_ra, dec = star.gaia_dec, unit=(u.degree, u.degree), frame='icrs')
width = u.Quantity(0.1, u.deg)
height = u.Quantity(0.1, u.deg)
r = Gaia.query_object_async(coordinate=coord, width=width, height=height)
r

SSLCertVerificationError: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1056)

In [66]:
ra0 = 0
dec0 = 0
fov = 10
limgmag = 18
job = Gaia.launch_job_async("SELECT * \
FROM gaiadr2.gaia_source \
WHERE CONTAINS(POINT('ICRS',gaiadr2.gaia_source.ra,gaiadr2.gaia_source.dec),CIRCLE('ICRS',%f,%f,%f))=1\
                            AND pmra IS NOT NULL AND abs(pmra)>0 \
 AND pmdec IS NOT NULL AND abs(pmdec)>0\
                            AND  phot_g_mean_mag<%f;" % (ra0, dec0, fov, limgmag) )#\
                            #, dump_to_file=False)
gaiat = job.get_results()

SSLCertVerificationError: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1056)

In [67]:
info.index

Index(['CMC15', 'Pan-STARRS DR1', '2MASS All-Sky', 'SDSS DR12', 'URAT1',
       'UCAC4'],
      dtype='object')

In [70]:
list(info.columns[:-2])

['ra', 'dec', 'ra_error', 'dec_error', 'ref_epoch', 'phot_g_mean_mag']

In [ ]:
['source_id']+info.columns[:-2]

In [73]:
astroinfo = ascii.read('cat_info.csv')
astroinfo

InconsistentTableError: 
ERROR: Unable to guess table format with the guesses listed below:
Reader:Ecsv fast_reader: {'enable': True} fill_values: [('', '0')] strict_names: True
Reader:FixedWidthTwoLine fast_reader: {'enable': True} fill_values: [('', '0')] strict_names: True
Reader:RST fast_reader: {'enable': True} fill_values: [('', '0')] strict_names: True
Reader:FastBasic fast_reader: {'enable': True} fill_values: [('', '0')] strict_names: True
Reader:Basic fast_reader: {'enable': True} fill_values: [('', '0')] strict_names: True
Reader:FastRdb fast_reader: {'enable': True} fill_values: [('', '0')] strict_names: True
Reader:Rdb fast_reader: {'enable': True} fill_values: [('', '0')] strict_names: True
Reader:FastTab fast_reader: {'enable': True} fill_values: [('', '0')] strict_names: True
Reader:Tab fast_reader: {'enable': True} fill_values: [('', '0')] strict_names: True
Reader:Cds fast_reader: {'enable': True} fill_values: [('', '0')] strict_names: True
Reader:Daophot fast_reader:

In [74]:
def GudeTable(gaiadf, catdf):
    gudedf = pd.DataFrame(columns=['k_g', 'e_g', 'k_c', 'e_c'])
    tg = 2015.5
    median_epoch = catdf.describe().ref_epoch['50%']
    for gaia_id in list(gaiadf.index):
        gstar = gaiadf.loc[gaia_id]
        pmr = gstar.pmra
        pmd = gstar.pmdec
        bias = 10000
        found = False
        for cat_id in list(catdf.index):
            cstar = catdf.loc[cat_id]
            t = cstar.ref_epoch
            k_c, e_c = tangFromRADE(cstar.ra * math.pi / 180., cstar.dec * math.pi / 180.,
                                    star.gaia_ra * math.pi / 180., star.gaia_dec * math.pi / 180.)  # [rad]
            # to take into account pm
            alpha, delta = RADecFromTang((t - tg) * math.pi * pmr / 648000000.0,
                                         (t - tg) * math.pi * pmd / 648000000.0,
                                         math.radians(gstar.ra), math.radians(gstar.dec))
            # rr, dd = math.degrees(alpha), math.degrees(delta)
            k_g, e_g = tangFromRADE(alpha, delta, star.gaia_ra * math.pi / 180.0,
                                    star.gaia_dec * math.pi / 180.0)  # [radians]
            biasK = abs(k_c - k_g) * 648000000.0 / math.pi  # [mas]
            biasE = abs(e_c - e_g) * 648000000.0 / math.pi  # [mas]
            current_bias = math.sqrt(biasE ** 2 + biasK ** 2)
            if current_bias < bias:
                bias = current_bias
                found = True
                drop_id = cat_id
                gude_star = [k_g * 648000000.0 / math.pi, e_g * 648000000.0 / math.pi,
                             k_c * 648000000.0 / math.pi, e_c * 648000000.0 / math.pi]
        if found:
            gudedf.loc[gaia_id] = gude_star
            catdf = catdf.drop(index=drop_id)
    return gudedf


In [4]:
import pandas as pd
import os

outdir = 'catalog_results'
catalog = 'URAT1'
outpath = os.path.join(outdir,catalog+'.csv')
result = pd.read_csv(outpath, index_col=0)
print(len(result))
result.sample(25)

1601


,gaia_ra,gaia_dec,cat_ra,cat_dec,dt,dm,dist_sec,gaia_pmra,gaia_pmdec,new_pmksi,new_pmeta,F
1821140080015765888,296.562868,18.036590,296.570813,18.035357,1.6,3.50,27.538,-4.434185,0.544723,79.409584,79.995155,6.130568
3979553453361445888,175.995225,21.459631,175.995221,21.459721,1.6,1.62,0.336,-7.353235,-187.857363,-13.870518,3.527208,48.183536
343775818097480192,25.478713,38.723467,25.478757,38.723552,2.1,2.01,0.320,-99.122745,-136.372140,-27.993080,6.162479,24.630075
1087155045521269504,111.195408,61.316938,111.195418,61.316938,1.8,1.97,0.021,-38.628867,14.295063,-34.748614,11.622447,0.862762
2848716674881969536,4.021479,23.318892,4.020842,23.318975,2.2,1.00,2.155,138.184623,-59.868402,-0.531152,2.837333,30.880374
2235993509403053952,296.828331,59.716570,296.817433,59.716064,1.9,0.97,19.882,150.795629,348.438082,13.562706,27.871624,80.153052
4353629578533517952,251.238955,-4.799835,251.238908,-4.799794,1.5,2.11,0.233,100.309234,-130.363719,-9.616925,-41.629848,12.925167
4277824333473792640,274.347926,3.117042,274.341839,3.112592,1.9,8.57,27.147,-25.641272,14.239895,141.059121,68.595284,14.090757
1170359243627867904,229.137932,12.956993,229.138011,12.956958,1.7,1.76,0.294,-154.150882,67.870196,14.435389,-16.236103,52.743303
2033400627906890496,297.906869,30.422978,297.907752,30.423780,1.9,5.10,3.952,-14.063851,-3.801693,117.008713,-136.816967,9.916678


In [13]:
result[result.dist_sec < 1].sort_values('F', ascending=False)

,gaia_ra,gaia_dec,cat_ra,cat_dec,dt,dm,dist_sec,gaia_pmra,gaia_pmdec,new_pmksi,new_pmeta,F
3957994474173323136,192.612327,25.167211,192.612560,25.167191,1.7,NaN,0.750,-458.668368,59.089397,4.482368,12.672277,166.452302
4256586888473586944,279.998518,-4.994580,279.998511,-4.994550,2.1,NaN,0.098,-6.557966,-35.061103,1022.877834,-2923.277859,145.985416
1493461352609320448,218.495972,44.298786,218.496115,44.298561,1.8,1.46,0.892,-171.559855,372.658018,-8.553965,-1.901912,124.181249
1467576173127694464,195.483246,33.589735,195.483491,33.589806,1.7,1.73,0.772,-423.092457,-152.646617,17.286129,-9.427403,122.394485
1309964514180794624,254.610270,31.075294,254.610477,31.075194,1.6,1.72,0.731,-379.387283,255.202358,-1.381820,20.061258,122.385356
...,...,...,...,...,...,...,...,...,...,...,...,...
391048594684815872,7.206766,48.029859,7.206674,48.029864,2.1,NaN,0.251,5.782510,-11.479538,-2.291155,-12.323287,1.073306
2072701159214477824,299.772954,39.557416,299.772986,39.557514,2.2,NaN,0.339,-16.116356,0.046529,-8.752673,-6.900285,1.068009
1087155045521269504,111.195408,61.316938,111.195418,61.316938,1.8,1.97,0.021,-38.628867,14.295063,-34.748614,11.622447,0.862762
1825536580280910464,293.967968,20.001185,293.967959,20.001188,1.9,NaN,0.056,-8.199168,-5.452795,-7.838471,-15.332802,0.716953
